<a href="https://colab.research.google.com/github/visahan1/Tensorflow/blob/main/Custom_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Custom VGG Network using Tensorflow

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D,MaxPool2D,GlobalMaxPool2D,Flatten,Dense

In [3]:
# Define a small class MyClass
class MyClass:
    def __init__(self):
        # One class variable 'a' is set to 1
        self.var1 = 1

# Create an object of type MyClass()
my_obj = MyClass()  

In [4]:
my_obj.__dict__

{'var1': 1}

In [5]:
# Add a new instance variable and give it a value
my_obj.var2 = 2

# Calls vars() again to see the object's instance variables
vars(my_obj)  

{'var1': 1, 'var2': 2}

In [6]:
my_obj.__dict__

{'var1': 1, 'var2': 2}

In [7]:
# Define a small class MyClass
class MyClass:
    def __init__(self):
        # Use vars(self) to access the class's dictionary of variables
        vars(self)['var1'] = 1

# Create an object of type MyClass()
my_obj = MyClass()
vars(my_obj)

{'var1': 1}

In [8]:
# Format a string using f-string notation
i=1
print(f"var{i}")

# Format a string using .format notation
i=2
print("var{}".format(i))

var1
var2


In [9]:
 class Block(tf.keras.Model):
     def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
         super(Block, self).__init__()
         self.filters = filters
         self.kernel_size = kernel_size
         self.repetitions = repetitions
        
         # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
         for i in range(self.repetitions):
            
             # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
             vars(self)[f'conv2D_{i}'] = Conv2D(filters=self.filters,kernel_size=self.kernel_size,padding='same',activation='relu')
        
         # Define the max pool layer that will be added after the Conv2D blocks
         self.max_pool = MaxPool2D()
  
     def call(self, inputs):
         # access the class's conv2D_0 layer
         conv2D_0 = vars(self)['conv2D_0'] 
        
         # Connect the conv2D_0 layer to inputs
         x = conv2D_0(inputs)

         # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
         for i in range(1,self.repetitions):
             # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
             conv2D_i = vars(self)[f'conv2D_{i}']
            
             # Use the conv2D_i and connect it to the previous layer
             x = conv2D_i(x)

         # Finally, add the max_pool layer
         max_pool = self.max_pool(x)
        
         return max_pool

## Create the Custom VGG network (TODO)
This model stack has a series of VGG blocks, which can be created using the `Block` class that you defined earlier.

### `__init__`
- Recall that the `__init__` constructor of `Block` takes several function parameters, 
    - filters, kernel_size, repetitions: you'll set these.
    - kernel_size and strides: you can use the default values.
- For blocks a through e, build the blocks according to the following specifications:
- block_a: 64  filters, kernel_size 3, repetitions 2
- block_b: 128 filters, kernel_size 3, repetitions 2
- block_c: 256 filters, kernel_size 3, repetitions 3
- block_d: 512 filters, kernel_size 3, repetitions 3
- block_e: 512 filters, kernel_size 3, repetitions 3

After block 'e', add the following layers:
- flatten: use [Flatten](https://keras.io/api/layers/reshaping_layers/flatten/).
- fc: create a fully connected layer using [Dense](https://keras.io/api/layers/core_layers/dense/).  Give this 256 units, and a `'relu'` activation.
- classifier: create the classifier using a Dense layer.  The number of units equals the number of classes.  For multi-class classification, use a `'softmax'` activation.

### `call`
Connect these layers together using the functional API syntax:
- inputs
- block_a
- block_b
- block_c
- block_d
- block_e
- flatten
- fc
- classifier

Return the classifier layer.

In [10]:
## Please uncomment all lines in this cell and replace those marked with `# YOUR CODE HERE`.
## You can select all lines in this code cell with Ctrl+A (Windows/Linux) or Cmd+A (Mac), then press Ctrl+/ (Windows/Linux) or Cmd+/ (Mac) to uncomment.



class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Creating blocks of VGG with the following 
        # (filters, kernel_size, repetitions) configurations
        self.block_a = Block(filters=64,kernel_size=3,repetitions=2)
        self.block_b = Block(filters=128,kernel_size=3,repetitions=2)
        self.block_c = Block(filters=256,kernel_size=3,repetitions=3)
        self.block_d = Block(filters=512,kernel_size=3,repetitions=3)
        self.block_e = Block(filters=512,kernel_size=3,repetitions=3)

        # Classification head
        # Define a Flatten layer
        self.flatten = Flatten()
        # Create a Dense layer with 256 units and ReLU as the activation function
        self.fc = Dense(units=256,activation='relu')
        # Finally add the softmax classifier using a Dense layer
        self.classifier = Dense(units=num_classes,activation='softmax')

    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [11]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes 
vgg = MyVGG(num_classes=2)

# Compile with losses and metrics
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
vgg.fit(dataset, epochs=10)

Shuffling and writing examples to data/cats_vs_dogs/4.0.0.incompleteK6660M/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to data/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Epoch 1/10
727/727 [==============================] - 141s 147ms/step - loss: 0.6865 - accuracy: 0.5550
Epoch 2/10
727/727 [==============================] - 104s 144ms/step - loss: 0.6605 - accuracy: 0.6157
Epoch 3/10
727/727 [==============================] - 105s 144ms/step - loss: 0.6334 - accuracy: 0.6473
Epoch 4/10
727/727 [==============================] - 105s 144ms/step - loss: 0.6129 - accuracy: 0.6703
Epoch 5/10
727/727 [==============================] - 105s 144ms/step - loss: 0.5988 - accuracy: 0.6808
Epoch 6/10
727/727 [==============================] - 105s 144ms/step - loss: 0.5886 - accuracy: 0.6885
Epoch 7/10
727/727 [==============================] - 105s 144ms/step - loss: 0.5807 - accuracy: 0.6946
Epoch 8/10
727/727 [==============================] - 105s 144ms/step - loss: 0.5740 - accuracy: 0.7005
Epoch 9/10
727/727 [==============================] - 10